En esta sección, se analizan la incrustación y las aplicaciones relacionadas con la incrustación. 
Comenzamos con un ejemplo para ilustrar qué es la incrustación, junto con varias formas de calcular la similitud vectorial (distancia euclidiana, similitud del producto del punto, similitud del coseno). 
Demostramos algunos casos de uso comunes de incrustación, como la búsqueda y recomendación, la clasificación, la agrupación en clústeres y la detección de valores atípicos. 
Después de eso, presentamos algunas formas de almacenar y buscar datos vectoriales, incluidos archivos planos, pgvector, OpenSearch Serverless y Pinecone.

Ahora puede hacer una pregunta y recibe el texto generado.
Cuando haces una pregunta de seguimiento, el texto generado parece no estar relacionado con la pregunta y la respuesta anteriores.
Cada pregunta parece ser tratada por separado: los modelos de fundación no parecen recordar lo que se ha preguntado y respondido previamente.

Al final de esta sección, podrá implementar la búsqueda en lenguaje natural en el servicio de chatbot conversacional sin servidor creado en la sección anterior.

La definición académica de incrustación es **traducir vectores de alta dimensión en un espacio de baja dimensión**. 

Es posible que conozcas todas y cada una de las palabras de esta oración, pero aún no tengas idea de toda la oración. Podemos pensar en la incrustación como la conversión del lenguaje natural en una secuencia de números, siendo la entrada un fragmento de texto y la salida un vector. En otras palabras, el vector es una representación numérica del texto, lo que facilita la realización de todo tipo de cálculos complejos en IA/ML.

In [4]:
import pandas as pd
import json
import boto3

bedrock = boto3.client(
    service_name='bedrock-runtime'
)

modelId = 'amazon.titan-embed-text-v1'
accept = 'application/json'
contentType = 'application/json'
prompt = """Quiero aprender AWS"""
input = {
        'inputText': prompt
    }
body=json.dumps(input)
response = bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept,contentType=contentType)
response_body = json.loads(response.get('body').read())
embedding = response_body['embedding']

#El modelo de base decide el número de elementos (dimensiones) en el vector, no el texto de entrada. Con el modelo amazon.titan-embed-text-v1, el tamaño del vector de salida es 1536.
print(len(embedding))


1536


**Distancia euclidiana**

Cuando se utilizan vectores para representar texto, se puede calcular la distancia euclidiana entre dos fragmentos de texto.

Aquí está el código de ejemplo para calcular la distancia euclidiana entre "hola" y "buen día".

In [9]:
import json
import boto3
import math

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

def calculate_distance(v1, v2):
    distance = math.dist(v1, v2)
    return distance

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)

text1 = 'AI'
text2 = 'ML'
v1 = get_embedding(bedrock, text1)
v2 = get_embedding(bedrock, text2)
distance = calculate_distance(v1, v2)
print(distance)


26.43179560150841


Quizás te preguntes por qué a la gente le importa "la distancia entre dos piezas de texto". ¿Qué significa realmente?

La distancia euclidiana entre dos fragmentos de texto indica la distancia a la que se encuentran entre sí. En términos legibles por humanos, significa qué tan similares son en el lenguaje natural. Si la distancia es muy pequeña, los dos fragmentos de texto transmiten un mensaje similar. Si la distancia es muy grande, los dos fragmentos de texto transmiten un mensaje diferente.

Reutilizando el código mencionado anteriormente, calculamos la distancia euclidiana entre "hola" y un montón de textos diferentes

In [77]:
word_comparative = "Xochimilco"

texts = [
    'lago',
    'trajinearas',
    'caidas',
    'mariachi',
    'palomas',
    "tequilas",
    'policia',
    'laptop'
]

In [78]:
# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
hello = get_embedding(bedrock, word_comparative)

texts_distance = []
for text in texts:
    embedding = get_embedding(bedrock, text)
    distance = calculate_distance(hello, embedding)
    texts_distance.append(distance)

distance_euclidian = pd.DataFrame(columns=[texts,texts_distance]).transpose().reset_index()
distance_euclidian.columns = ["words","distance_euclidian"]

distance_euclidian['Rank_euclidian'] = distance_euclidian.sort_values(by=['distance_euclidian'], ascending=True) \
                                                         .reset_index() \
                                                         .sort_values('index') \
                                                         .index + 1

distance_euclidian

,words,distance_euclidian,Rank_euclidian
0,lago,27.620055,7
1,trajinearas,26.271369,4
2,caidas,26.468978,5
3,mariachi,25.565576,3
4,palomas,24.265096,2
5,tequilas,22.818800,1
6,policia,27.307918,6
7,laptop,33.889377,8


**Similitud del producto Dot**

La distancia euclidiana no es la única forma de medir la similitud entre dos vectores. La similitud de productos escalar es otro método comúnmente utilizado para medir la similitud. En álgebra lineal, el producto escalar entre dos vectores mide el grado en que dos vectores se alinean en la misma dirección. Si el producto escalar de dos vectores es 0, los dos vectores son ortogonales (perpendiculares), que es una especie de similitud intermedia.

Aquí está el código de ejemplo para calcular la similitud del producto punto entre "hola" y "buen día".

In [79]:
import json
import boto3
from numpy import dot
from numpy.linalg import norm

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

def calculate_dot_product_similarity(v1, v2):
    similarity = dot(v1, v2)
    return similarity

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
text1 = 'hello'
text2 = 'good day'
v1 = get_embedding(bedrock, text1)
v2 = get_embedding(bedrock, text2)
similarity = calculate_dot_product_similarity(v1, v2)
print(similarity)


245.3465466016232


Si el ángulo entre dos vectores es menor que 90 grados, los dos vectores están aproximadamente en la misma dirección, entonces el producto escalar es positivo. Si el ángulo entre dos vectores es mayor que 90 grados, los dos vectores están aproximadamente en direcciones opuestas, entonces el producto escalar es negativo. Cuanto más grande sea el producto escalonado, es más probable que los dos vectores estén alineados en la misma dirección.

Reutilizando el código mencionado anteriormente, calculamos la similitud del producto punto entre "hola" y un montón de textos diferentes.

In [80]:
# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
hello = get_embedding(bedrock, word_comparative)

texts_similarity = []
for text in texts:
    embedding = get_embedding(bedrock, text)
    similarity = calculate_dot_product_similarity(hello, embedding)
    texts_similarity.append(similarity)

distance_dot_product = pd.DataFrame(columns=[texts,texts_similarity]).transpose().reset_index()
distance_dot_product.columns = ["words","similarity_dot_product"]

distance_dot_product['Rank_dot_product'] = distance_dot_product.sort_values(by=['similarity_dot_product'], ascending=True) \
                                                             .reset_index() \
                                                             .sort_values('index') \
                                                             .index + 1

distance_dot_product

,words,similarity_dot_product,Rank_dot_product
0,lago,154.109042,5
1,trajinearas,115.263363,2
2,caidas,165.162233,6
3,mariachi,175.569933,7
4,palomas,144.825127,4
5,tequilas,240.368924,8
6,policia,139.765334,3
7,laptop,46.699242,1


**Similitud de coseno**

La similitud del coseno es otro método comúnmente utilizado para medir la similitud. En álgebra lineal, la similitud del coseno es el coseno del ángulo entre dos vectores. Es decir, es el producto escalar de los vectores dividido por el producto de sus longitudes.

Aquí está el código de ejemplo para calcular la similitud del coseno entre "hola" y "buen día".

In [81]:
import json
import boto3
from numpy import dot
from numpy.linalg import norm

def get_embedding(bedrock, text):
    modelId = 'amazon.titan-embed-text-v1'
    accept = 'application/json'
    contentType = 'application/json'
    input = {
            'inputText': text
        }
    body=json.dumps(input)
    response = bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept,contentType=contentType)
    response_body = json.loads(response.get('body').read())
    embedding = response_body['embedding']
    return embedding

def calculate_cousin_similarity(v1, v2):
    similarity = dot(v1, v2)/(norm(v1)*norm(v2))
    return similarity

# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
text1 = 'hello'
text2 = 'good day'
v1 = get_embedding(bedrock, text1)
v2 = get_embedding(bedrock, text2)
similarity = calculate_cousin_similarity(v1, v2)
print(similarity)


0.47547669109839213


Al igual que la distancia euclidiana, la similitud del coseno mide la similitud de dos textos en cuanto a su tema, que es independiente de la longitud de los textos. Cuando la similitud del coseno es igual a 1, significa que dos fragmentos de texto tienen significados idénticos en el lenguaje natural.

Reutilizando el código mencionado anteriormente, calculamos la similitud del coseno entre "hola" y un montón de textos diferentes.

In [82]:
# main function
bedrock = boto3.client(
    service_name='bedrock-runtime'
)
hello = get_embedding(bedrock, word_comparative)

text_cousins_similarity = []
for text in texts:
    embedding = get_embedding(bedrock, text)
    similarity = calculate_cousin_similarity(hello, embedding)
    text_cousins_similarity.append(similarity)

distance_cousin_similarity = pd.DataFrame(columns=[texts,text_cousins_similarity]).transpose().reset_index()
distance_cousin_similarity.columns = ["words","similarity_cousins_similarity"]

distance_cousin_similarity['Rank_cousins_similarity'] = distance_cousin_similarity.sort_values(by=['similarity_cousins_similarity'], ascending=True) \
                                                                      .reset_index() \
                                                                      .sort_values('index') \
                                                                      .index + 1

distance_cousin_similarity

,words,similarity_cousins_similarity,Rank_cousins_similarity
0,lago,0.289204,4
1,trajinearas,0.250660,2
2,caidas,0.321085,5
3,mariachi,0.349767,7
4,palomas,0.331316,6
5,tequilas,0.480379,8
6,policia,0.273129,3
7,laptop,0.077159,1


**Discusiones**

En la tabla siguiente se comparan los resultados producidos por la distancia euclidiana, la similitud del producto escalar y la similitud del coseno. Los números de la tabla representan la posición de cada palabra, ordenada por la similitud entre la palabra y "hola".

In [83]:
print(word_comparative)
print("Diferentes similitudes :")
pd.concat([distance_euclidian,distance_dot_product,distance_cousin_similarity], axis = 1).iloc[:,[0,2,5,8]]

Xochimilco
Diferentes similitudes :


,words,Rank_euclidian,Rank_dot_product,Rank_cousins_similarity
0,lago,7,5,4
1,trajinearas,4,2,2
2,caidas,5,6,5
3,mariachi,3,7,7
4,palomas,2,4,6
5,tequilas,1,8,8
6,policia,6,3,3
7,laptop,8,1,1
